In [1]:
import os
import geopandas as gpd
from glob import glob
import numpy as np
import pandas as pd 
import rasterio 
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
import shutil
import sys
import xarray as xr
from osgeo import gdal, osr
import os.path


home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"
exec(open(os.path.join(home, "Scripts", "Functions.py")).read())

# python -m pip install "xarray[complete]"
# https://colab.research.google.com/drive/1B7gFBSr0eoZ5IbsA0lY8q3XL8n-3BOn4#scrollTo=1I_gUeqs5Sak

In [35]:
# calculate vpd 
def calc_vpd(tmin, tmax, vp):
    vp_kpa = vp * 0.001
    tavg = (tmin + tmax)/2
    vpd=(0.61078 * np.exp(17.26939 * tavg / (tavg + 237.3))) - vp_kpa
    return vpd 
vpd_vectorized = np.vectorize(calc_vpd)

# order the files by year 
#def sort_files(files):
#    names = [os.path.basename(x) for x in files]
#    file_year = [int(x[-13:-9]) for x in names]
#    files = [x for _, x in sorted(zip(file_year, files))]  
#    return files

def vpd(vp_file):
    
    names = os.path.basename(vp_file)
    file_year = str(names[-13:-9])

    # check if the file already exists 
    if os.path.exists(os.path.join(home, "Data", "Climate", "vpd_" + str(file_year) + "subset.nc")) == True:
        return

    tmin_file = glob(os.path.join(home, "Data", "Climate", "tmin_" + file_year+ "subset.nc"))
    tmax_file = glob(os.path.join(home, "Data", "Climate", "tmax_" + file_year+ "subset.nc"))

    tmin_ds = xr.open_mfdataset(tmin_file)
    tmax_ds = xr.open_mfdataset(tmax_file)
    vp_ds = xr.open_mfdataset(vp_file)

    tmin_np = tmin_ds['tmin'].to_numpy()
    tmax_np = tmax_ds['tmax'].to_numpy()
    vp_np = vp_ds['vp'].to_numpy()
    vpd_np = vpd_vectorized(tmin_np, tmax_np, vp_np)
    tmin_ds['vpd'] = (['time', 'y', 'x'], vpd_np)
    vpd_ds = tmin_ds['vpd']
    vpd_ds.to_netcdf(path = os.path.join(home, "Data", "Climate", "vpd_" + str(file_year) + "subset.nc"))
    
    tmin_ds.close()
    tmax_ds.close()
    vp_ds.close()
    vpd_ds.close()

    return

vp_files = glob(os.path.join(home, "Data", "Climate", "vp_" + "*.nc"))
for vp_file in vp_files:
    vpd(vp_file)
    print(vp_file)


/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1986subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1987subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1988subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1989subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1990subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1991subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1992subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1993subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1995subset.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/vp_1996

In [5]:
# calculate PAR from short wave radiation 
# SWRAD * 2.07 
# following 'Modeling Gross Primary Production of Midwestern US Maize and Soybean Croplands with Satellite and Gridded Weather Data'


# calculate daily swrad following daymet website instructions: ((srad (W/m2) * dayl (s/day)) = Daily total radiation (J/m2/day)
# Then J/m2/day * 2.07 = PAR in micromoles m-2 d-1
# Then convert to mol m-2 d-1

def calc_par(swrad, dayl):
    daily_total_radiation = swrad * dayl # srad (W/m2) * dayl (s/day) = Daily total radiation (J/m2/day) 
    par = (daily_total_radiation*2.07)/1000000
    return par
par_vectorized = np.vectorize(calc_par)

def par(sw_file):
    names = os.path.basename(sw_file)
    file_year = str(names[-13:-9])
    dayl_file = glob(os.path.join(home, "Data", "Climate", "dayl_" + file_year + "subset.nc"))

    # check if the file already exists 
    if os.path.exists(os.path.join(home, "Data", "Climate", "par_" + str(file_year) + "subset.nc")) == True:
        return
    
    sw_ds = xr.open_mfdataset(sw_file)
    sw_np = sw_ds['srad'].to_numpy()

    dayl_ds = xr.open_mfdataset(dayl_file)
    dayl_np = dayl_ds['dayl'].to_numpy()

    par_np = par_vectorized(sw_np, dayl_np)

    sw_ds['par'] = (['time', 'y', 'x'], par_np)
    par_ds = sw_ds['par']
    par_ds.to_netcdf(path = os.path.join(home, "Data", "Climate", "par_" + str(file_year) + "subset.nc"))
    return par_ds

sw_files = glob(os.path.join(home, "Data", "Climate", "srad_*subset.nc"))
for sw_file in sw_files:
    par(sw_file)
    print(sw_file)

<xarray.DataArray 'par' (time: 365, y: 494, x: 559)>
array([[[16.00477081, 16.00826034, 16.04245597, ..., 19.03591652,
         19.03732796, 19.01897066],
        [16.0626935 , 16.08432706, 16.10177361, ..., 19.04580304,
         19.04227122, 19.03238685],
        [16.11782521, 16.10875266, 16.11293966, ..., 19.04438945,
         19.04015299, 19.0210889 ],
        ...,
        [20.92343461, 20.57869641, 20.43657242, ..., 14.19043606,
         14.06094982, 18.55689307],
        [20.88553591, 20.85346693, 20.84253483, ..., 14.12642825,
         13.99032198, 13.84906519],
        [20.90553326, 20.91355343, 20.91282574, ..., 14.07860734,
         13.94102934, 13.8027149 ]],

       [[ 7.9927391 ,  8.03675486,  7.88514447, ..., 13.92117097,
         13.96711733, 13.95863519],
        [ 7.8061954 ,  7.77964576,  7.77475511, ..., 13.92541204,
         13.96570436, 13.97842756],
        [ 7.6343231 ,  7.81108605,  7.93195496, ..., 13.89501698,
         13.93884281, 13.92965311],
...
        [10.86764096, 10.85235955, 10.85890825, ..., 19.47987951,
         19.46004636, 19.43800729],
        [11.00444076, 11.26421683, 11.47087274, ..., 19.46298535,
         19.43800729, 19.41523516],
        [11.02168035, 11.07919441, 11.08720281, ..., 19.46151474,
         19.4387426 , 19.41670577]],

       [[10.98105945, 10.88834215, 10.74891784, ..., 13.26855703,
         13.21071379, 13.2579754 ],
        [10.92040915, 10.82141801, 10.72521596, ..., 13.22200043,
         13.1662733 , 13.18814158],
        [10.85975991, 10.81584091, 10.75100912, ..., 13.23117097,
         13.17826604, 13.21847322],
        ...,
        [14.96710923, 14.82583434, 14.71295999, ..., 21.04660015,
         21.09952917, 21.162751  ],
        [15.27223366, 15.77761865, 16.17522624, ..., 21.13996234,
         21.19803552, 21.25243623],
        [15.25883165, 15.29161903, 15.25373211, ..., 21.19803552,
         21.25611095, 21.3046294 ]]])
Coordinates:
  * y        (y) float32 -307.0 -308.0 -309.0 -310.0 ... -798.0 -799.0 -800.0
  * x        (x) float32 1.256e+03 1.257e+03 1.258e+03 ... 1.813e+03 1.814e+03
  * time     (time) datetime64[ns] 2012-01-01T12:00:00 ... 2012-12-30T12:00:00

In [7]:
def growing_season_anomalies(VAR, agg):
    
    files = glob(os.path.join(home, "Data", "Climate", VAR + "_" + "*.nc"))
    # order the files by year 
    names = [os.path.basename(x) for x in files]
    file_year = [int(x[-13:-9]) for x in names]
    files = [x for _, x in sorted(zip(file_year, files))]   
    K = -1 
    for file in files:
        print(file)
        K += 1
        ds = xr.open_mfdataset(file)[VAR]
        
        if agg == 'mean':
            if K == 0:
                ds_amjja = ds.sel(time = np.isin(ds.time.dt.month, [4,5,6,7,8])).mean('time')
            else:
                ds_amjja = xr.concat([ds_amjja, ds.sel(time = np.isin(ds.time.dt.month, [4,5,6,7,8])).mean('time')], dim='year')
        if agg == 'sum':
            if K == 0:
                ds_amjja = ds.sel(time = np.isin(ds.time.dt.month, [4,5,6,7,8])).sum('time')
            else:
                ds_amjja = xr.concat([ds_amjja, ds.sel(time = np.isin(ds.time.dt.month, [4,5,6,7,8])).sum('time')], dim='year')
    
    ds_amjja_anomalies = (ds_amjja - ds_amjja.mean('year'))/ds_amjja.std('year')
    ds_amjja_climatology = ds_amjja.mean('year')

    ds_amjja_anomalies.to_netcdf(path = os.path.join(home, "Data", "Climate", "anomalies", VAR + "_AMJJA.nc"))
    ds_amjja_climatology.to_netcdf(path = os.path.join(home, "Data", "Climate", "climatology", VAR + "_AMJJA.nc"))

In [ ]:
growing_season_anomalies("tmin", "mean")
growing_season_anomalies("tmax", "mean")
growing_season_anomalies("prcp", "sum")
growing_season_anomalies("vp", "mean")
growing_season_anomalies("vpd", "mean")
growing_season_anomalies("par", "mean")

In [12]:
# convert the .nc files to tifs with one for each year 
# convert nc files to tifs 
def nc_to_tif(nc, template):
    # nc is the netcdf filepath i want to write to tifs
    # ds is the template raster filepath that i will write the nc files to match 

    # template 
    ds = gdal.Open(template)
    wkt = ds.GetProjection()
    trans = ds.GetGeoTransform()
    cols = ds.RasterXSize
    rows = ds.RasterYSize   
    
    mync = xr.open_dataset(nc)
    myarray = mync.to_array()
    myarray = myarray.to_numpy()
    if len(myarray.shape)==4:
        myarray = myarray[0,:,:,:]
    
    K = 1983
    for time in range(0, myarray.shape[0]):
        K +=1
        myslice = myarray[time,:,:]

        # create the output image
        if len(range(0, myarray.shape[0])) == 1:
            breakdown_path = nc.split("/")
            outname = breakdown_path[len(breakdown_path)-2] + "_" + breakdown_path[len(breakdown_path)-1][:-3] + ".tif"
            outpath = os.path.join(home, "Data", "Climate", "tifs", outname)
        else:
            breakdown_path = nc.split("/")
            outname = breakdown_path[len(breakdown_path)-2] + "_" + breakdown_path[len(breakdown_path)-1][:-3] + "_" + str(K) + ".tif"
            outpath = os.path.join(home, "Data", "Climate", "tifs", outname)

        driver = ds.GetDriver()
        outDs = driver.Create(outpath, cols, rows, 1, gdal.GDT_Float32)
        outBand = outDs.GetRasterBand(1)
        outBand.WriteArray(myslice)
        outDs.SetGeoTransform(trans)    
        srs = osr.SpatialReference()
        srs.ImportFromWkt(wkt)
        outDs.SetProjection(srs.ExportToWkt())

        outDs = None

In [13]:
# make a raster template 
# template = rxr.open_rasterio('/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/prcp_1984subset.nc')
# template['prcp'].rio.to_raster('/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/template.tif')
template = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/template.tif"
climatology_nc = glob(os.path.join(home, "Data", "Climate", "climatology", "*AMJJA.nc"))
anomalies_nc = glob(os.path.join(home, "Data", "Climate", "anomalies", "*AMJJA.nc"))
all_nc = climatology_nc + anomalies_nc

for nc in all_nc:
    nc_to_tif(nc, template)

### GridMET VPD

In [37]:
files = glob(os.path.join(home, "Data", "Climate", "gridmet", "vpd", "vpd_" + "*.nc"))
names = [os.path.basename(x) for x in files]
file_year = [int(x[-7:-3]) for x in names]
files = [x for _, x in sorted(zip(file_year, files))]

K = -1
for file in files:
    print(file)
    K +=1
    ds = xr.open_mfdataset(file)['mean_vapor_pressure_deficit']
    if K == 0:
        ds_amjja = ds.sel(day = np.isin(ds.day.dt.month, [4,5,6,7,8])).mean('day')
    else:
        ds_amjja = xr.concat([ds_amjja, ds.sel(day = np.isin(ds.day.dt.month, [4,5,6,7,8])).mean('day')], dim='year')

ds_amjja_anomalies = (ds_amjja - ds_amjja.mean('year'))/ds_amjja.std('year')
ds_amjja_climatology = ds_amjja.mean('year')

ds_amjja_anomalies.to_netcdf(path = os.path.join(home, "Data", "Climate", 'gridmet',"anomalies", "vpd" + "_AMJJA.nc"))
ds_amjja_climatology.to_netcdf(path = os.path.join(home, "Data", "Climate", 'gridmet',"climatology", "vpd" + "_AMJJA.nc"))


/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1984.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1985.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1986.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1987.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1988.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1989.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1990.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1991.nc
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1992.nc
/Volumes/GoogleDrive/My Driv

/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [16]:
def gridmet_nc_to_tif(nc, template):
    
    # template 
    ds = gdal.Open(template)
    wkt = ds.GetProjection()
    trans = ds.GetGeoTransform()
    cols = ds.RasterXSize
    rows = ds.RasterYSize   
    
    mync = xr.open_dataset(nc)
    myarray = mync.to_array()
    myarray = myarray.to_numpy()
    if len(myarray.shape)==4:
        myarray = myarray[0,:,:,:]
    
    K = 1983
    for time in range(0, myarray.shape[0]):
        K +=1
        myslice = myarray[time,:,:]

        # create the output image
        if len(range(0, myarray.shape[0])) == 1:
            breakdown_path = nc.split("/")
            outname = breakdown_path[len(breakdown_path)-2] + "_" + breakdown_path[len(breakdown_path)-1][:-3] + ".tif"
            outpath = os.path.join(home, "Data", "Climate", 'gridmet',"tifs", outname)
        else:
            breakdown_path = nc.split("/")
            outname = breakdown_path[len(breakdown_path)-2] + "_" + breakdown_path[len(breakdown_path)-1][:-3] + "_" + str(K) + ".tif"
            outpath = os.path.join(home, "Data", "Climate", 'gridmet',"tifs", outname)

        driver = ds.GetDriver()
        outDs = driver.Create(outpath, cols, rows, 1, gdal.GDT_Float32)
        outBand = outDs.GetRasterBand(1)
        outBand.WriteArray(myslice)
        outDs.SetGeoTransform(trans)    
        srs = osr.SpatialReference()
        srs.ImportFromWkt(wkt)
        outDs.SetProjection(srs.ExportToWkt())

        outDs = None

In [14]:
nc = all_nc[0]

ds = gdal.Open(template)
wkt = ds.GetProjection()
trans = ds.GetGeoTransform()
cols = ds.RasterXSize
rows = ds.RasterYSize   
    
mync = xr.open_dataset(nc)
myarray = mync.to_array()
myarray = myarray.to_numpy()
if len(myarray.shape)==4:
    myarray = myarray[0,:,:,:]


In [17]:
# make a raster template 
import rioxarray as rxr
#template = rxr.open_rasterio('/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/vpd/vpd_1984.nc')
#template = template.sel(day="1984-01-01")
#template.rio.to_raster('/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/template.tif')
template = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/gridmet/template.tif"
climatology_nc = glob(os.path.join(home, "Data", "Climate", 'gridmet',"climatology", "*AMJJA.nc"))
anomalies_nc = glob(os.path.join(home, "Data", "Climate", 'gridmet',"anomalies", "*AMJJA.nc"))
all_nc = climatology_nc + anomalies_nc

for nc in all_nc:
    gridmet_nc_to_tif(nc, template)